In [ ]:
# Import required packages
import time
import numpy as np
import pandas as pd
from tabulate import tabulate
import torch
from tqdm import trange
from paik.solver import Solver
from paik.settings import (
    DEFAULT_NSF,
    DEFULT_SOLVER,
)

In [ ]:
WORKDIR = "/home/luca/paik"

solver_param = DEFULT_SOLVER
solver_param.workdir = WORKDIR
solver = Solver(solver_param=solver_param)

num_seeds = 2000
num_sols = num_seeds
num_poses = 1000
base_stds = np.arange(0.1, 1.5, 0.1)  # start, stop, step

In [ ]:
def get_numerical_ik_sols(target_pose, num_seeds):
    seeds, _ = solver._robot.sample_joint_angles_and_poses(
        n=num_seeds, return_torch=False
    )
    numer_ik = np.empty((num_seeds, solver.n))
    for i in range(num_seeds):
        numer_ik[i] = solver.robot.inverse_kinematics_klampt(
            pose=target_pose, seed=seeds[i]
        )
    return numer_ik


_, P = solver._robot.sample_joint_angles_and_poses(n=num_poses, return_torch=False)

J_ground_truth = np.empty((num_poses, num_sols, solver.n))
for i in trange(num_poses):
    J_ground_truth[i] = get_numerical_ik_sols(P[i], num_seeds)

In [ ]:
from example.mmd import mmd_evaluate_multiple_poses

l2_mean = np.empty((len(base_stds)))
ang_mean = np.empty((len(base_stds)))
mmd_mean = np.empty((len(base_stds)))

for i, std in enumerate(base_stds):
    solver.base_std = std
    P_expand_dim = (
        np.expand_dims(P, axis=1).repeat(num_sols, axis=1).reshape(-1, P.shape[-1])
    )

    F = solver.F[
        solver.P_knn.kneighbors(
            np.atleast_2d(P), n_neighbors=num_sols, return_distance=False
        ).flatten()
    ]
    J_hat = solver.solve_batch(P_expand_dim, F, 1)
    l2, ang = solver.evaluate_pose_error_J3d_P2d(J_hat, P_expand_dim, return_all=True)
    J_hat = J_hat.reshape(num_poses, num_sols, -1)
    l2_mean[i] = l2.mean()
    ang_mean[i] = ang.mean()
    mmd_mean[i] = mmd_evaluate_multiple_poses(
        J_hat, J_ground_truth, num_poses=num_poses
    )
    assert not np.isnan(mmd_mean[i])

df = pd.DataFrame({"l2": l2_mean, "ang": ang_mean, "mmd": mmd_mean, "std": base_stds})
df.plot(x="std", y=["l2"], title="l2 vs. Base Std", grid=True)
df.plot(x="std", y=["ang"], title="ang vs. Base Std", grid=True)
df.plot(x="std", y=["mmd"], title="mmd vs. Base Std", grid=True)
df.describe()

In [ ]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

set_seed()
# Build IKFlowSolver and set weights
ik_solver, _ = get_ik_solver("panda__full__lp191_5.25m")
l2_flow = np.empty((len(base_stds)))
ang_flow = np.empty((len(base_stds)))
mmd_flow = np.empty((len(base_stds)))

for i, std in enumerate(base_stds):
    J_flow = np.array(
        [ik_solver.solve(p, n=num_sols, latent_scale=std).cpu().numpy() for p in P]
    )
    J_flow = J_flow.transpose(1, 0, 2)  # (num_sols, num_poses, n)
    l2, ang = solver.evaluate_pose_error_J3d_P2d(J_flow, P, return_all=True)
    l2_flow[i] = l2.mean()
    ang_flow[i] = ang.mean()
    mmd_flow[i] = mmd_evaluate_multiple_poses(
        J_flow.transpose(1, 0, 2), J_ground_truth, num_poses=num_poses
    )


df_flow = pd.DataFrame(
    {"l2": l2_flow, "ang": ang_flow, "mmd": mmd_flow, "std": base_stds}
)
df_flow.plot(x="std", y=["l2"], title="l2 vs. Base Std", grid=True)
df_flow.plot(x="std", y=["ang"], title="ang vs. Base Std", grid=True)
df_flow.plot(x="std", y=["mmd"], title="mmd vs. Base Std", grid=True)
df_flow.describe()

In [ ]:
df_l2 = pd.DataFrame({"l2_paik": l2_mean, "l2_ikflow": l2_flow, "std": base_stds})
df_ang = pd.DataFrame({"ang_paik": ang_mean, "ang_ikflow": ang_flow, "std": base_stds})
df_mmd = pd.DataFrame({"mmd_paik": mmd_mean, "mmd_ikflow": mmd_flow, "std": base_stds})

df_l2.plot(x="std", y=["l2_paik", "l2_ikflow"], title="l2 vs. Base Std", grid=True)
df_ang.plot(x="std", y=["ang_paik", "ang_ikflow"], title="ang vs. Base Std", grid=True)
df_mmd.plot(x="std", y=["mmd_paik", "mmd_ikflow"], title="mmd vs. Base Std", grid=True)

In [ ]:
import os
from datetime import datetime

record_dir = f"{WORKDIR}/record/{datetime.today().strftime('%Y_%m_%d')}"
os.makedirs(record_dir, exist_ok=True)
df.to_pickle(f"{record_dir}/paik_posture_mmd_std.pkl")
df_flow.to_pickle(f"{record_dir}/ikflow_posture_mmd_std.pkl")

In [ ]:
import numpy as np

batch_size = 20
arr = np.arange(301)
brr = np.split(arr, np.arange(batch_size, arr.shape[0], batch_size))

In [ ]:
brr